In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import cv2
import os

dataset_path = os.listdir('D:/Deep_Learning/Dog_Rabies_Project/videos/train')
label_types = os.listdir('D:/Deep_Learning/Dog_Rabies_Project/videos/train')
print(label_types)

['Barking', 'Digging', 'Dropped Jaw & Tongue', 'Hyper Salivation', 'Incoordination', 'No Detection', 'Paralysis', 'Playing', 'Running', 'Seizure', 'Wagging Tail']


In [2]:
# Dataset Preperation
rooms = []

for item in dataset_path:
    all_rooms = os.listdir('D:/Deep_Learning/Dog_Rabies_Project/videos/train' + '/' + item)# Get all file names
    
    for room in all_rooms:# Add them to the list
        rooms.append((item, str('D:/Deep_Learning/Dog_Rabies_Project/videos/train' + '/' + item) + '/' + room))

# Build DataFrame
train_df = pd.DataFrame(data=rooms, columns = ['tag','video_name'])   
print(train_df.head())
print(train_df.tail())

       tag                                         video_name
0  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
1  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
2  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
3  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
4  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
              tag                                         video_name
438  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
439  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
440  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
441  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...
442  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tra...


In [3]:
df = train_df.loc[:, ['video_name', 'tag']]
df.to_csv('train1.csv')

In [4]:
dataset_path = os.listdir('D:/Deep_Learning/Dog_Rabies_Project/videos/test')
print(dataset_path)

room_types = os.listdir('D:/Deep_Learning/Dog_Rabies_Project/videos/test')
print('Types of behaviour found',len(dataset_path))

rooms = []

for item in dataset_path:
    all_rooms = os.listdir('D:/Deep_Learning/Dog_Rabies_Project/videos/test' + '/' + item)# Get all file names
    
    for room in all_rooms:# Add them to the list
        rooms.append((item, str('D:/Deep_Learning/Dog_Rabies_Project/videos/test' + '/' + item) + '/' + room))

# Build DataFrame
test_df = pd.DataFrame(data=rooms, columns = ['tag','video_name'])   
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:, ['video_name', 'tag']]
df.to_csv('test1.csv')

['Barking', 'Digging', 'Dropped Jaw & Tongue', 'Hyper Salivation', 'Incoordination', 'No Detection', 'Paralysis', 'Playing', 'Running', 'Seizure', 'Wagging Tail']
Types of behaviour found 11
       tag                                         video_name
0  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
1  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
2  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
3  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
4  Barking  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
              tag                                         video_name
146  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
147  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
148  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
149  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...
150  Wagging Tail  D:/Deep_Learning/Dog_Rabies_Project/videos/tes...


In [5]:
train_df = pd.read_csv('train1.csv')
test_df = pd.read_csv('test1.csv')

print(f'Total videos for training: {len(train_df)}')
print(f'Total videos for testing: {len(test_df)}')

train_df.sample(5)

Total videos for training: 443
Total videos for testing: 151


,Unnamed: 0,video_name,tag
304,304,D:/Deep_Learning/Dog_Rabies_Project/videos/tra...,Playing
44,44,D:/Deep_Learning/Dog_Rabies_Project/videos/tra...,Digging
368,368,D:/Deep_Learning/Dog_Rabies_Project/videos/tra...,Seizure
211,211,D:/Deep_Learning/Dog_Rabies_Project/videos/tra...,Incoordination
199,199,D:/Deep_Learning/Dog_Rabies_Project/videos/tra...,Incoordination


In [6]:
IMG_SIZE = 224

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1,0]]
            frames.append(frame)
            
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [7]:
# Feature Extraction
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
    weights = 'imagenet',
    include_top = False,
    pooling = 'avg',
    input_shape = (IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input
    
    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)
    
    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name='feature_extractor')

feature_extractor = build_feature_extractor()

In [8]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df['tag']))
print(label_processor.get_vocabulary())

labels = train_df['tag'].values
labels = label_processor(labels[..., None]).numpy()
labels

['Barking', 'Digging', 'Dropped Jaw & Tongue', 'Hyper Salivation', 'Incoordination', 'No Detection', 'Paralysis', 'Playing', 'Running', 'Seizure', 'Wagging Tail']


array([[ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],

In [9]:
IMG_SIZE = 224
BATCH_SIZE = 64

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [10]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df['video_name'].values.tolist()
    
    labels = df['tag'].values
    
    labels = label_processor(labels[..., None]).numpy()
    
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype='bool')
    frame_features =  np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype='float32')
    
    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH), dtype='bool')
        temp_frame_features =  np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype='float32')

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1 # 1 = not masked, 0 = masked
        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

In [11]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 100
# Utility for running experiments.
def run_experiment():
#     filepath = "./tmp/rabies_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
    )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

In [16]:
#save model structure in json file
model_json = sequence_model.to_json()
with open('model2.json','w') as json_file:
    json_file.write(model_json)

In [17]:
#save trained model weight in .h5 file
sequence_model.save_weights('model2.h5')

In [12]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask

In [13]:
def Video_play():   
    cap = cv2.VideoCapture(test_video)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file.")

    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            face_detector = cv2.CascadeClassifier('Dog_Cascade.xml')
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

            for (x, y, w, h) in num_faces:
                if Rab > Nor:
                    cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 0, 255), 4)
                    cv2.putText(frame, f"Rabies", (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, 
                                cv2.LINE_4)
                elif Nor > Rab:
                    cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
                    cv2.putText(frame, f"Normal", (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2,
                                cv2.LINE_4)
                else:
                    cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 4)
                    
            cv2.imshow('Rabies Detection', frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

In [31]:
Rabies = ['Paralysis','Dropped Jaw & Tongue','Hyper Salivation','Incoordination','Seizure',]
Normal = ['Barking','Running','Playing','Wagging Tail','Digging']
Rab = 0
Nor = 0
No_Detect = 0

def sequence_prediction(path):
    global Rab
    global Nor
    global No_Detect
    count = 0
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]
    
    print('Details:')
    for i in np.argsort(probabilities)[::-1]:
        if count < len(Rabies):
            if class_vocab[i] == 'No Detection':
                No_Detect += 1
            print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
            if class_vocab[i] in Rabies:
                Rab += 1
            else:
                Nor += 1
        count += 1
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

if Rab > Nor:
    print(f'Since Features of Rabies are more dominating among the features shown, the Final Result can be assumed as RABIES.')
elif Nor > Rab:
    print(f'Since Features of Normal are more dominating among the features shown, the Final Result can be assumed as NORMAL.')
else:
    print('No Dog Detected.')

play_video = Video_play()
# End of the Program.

Test video path: D:/Deep_Learning/Dog_Rabies_Project/videos/test/Paralysis/030.mp4
1/1 [==============================] - 0s 15ms/step
Details:
  Paralysis: 96.31%
  Incoordination:  2.21%
  Dropped Jaw & Tongue:  0.99%
  Barking:  0.49%
  No Detection:  0.00%
Since Features of Rabies are more dominating among the features shown, the Final Result can be assumed as RABIES.


In [32]:
# import numpy as np
# import cv2
# import time

# def live_capture():
#     capture_duration = 9 # The duration in seconds of the video captured.

#     cap = cv2.VideoCapture(0)

#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter('output.mp4',fourcc, 25, (640,480))

#     start_time = time.time()
#     while( int(time.time() - start_time) < capture_duration ):
#         ret, frame = cap.read()
#         if ret==True:
#             out.write(frame)
#             cv2.imshow('frame',frame)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#         else:
#             break

#     cap.release()
#     out.release()
#     cv2.destroyAllWindows()

In [ ]:
# import tkinter

# def UploadAction(event=None):
#     test_video = filedialog.askopenfilename()
#     print('Selected:', test_video)

# root = tk.Tk()
# button = tk.Button(root, text='Upload', command=UploadAction)
# button.pack()

# root.mainloop()